<a href="https://colab.research.google.com/github/Kiran-Venkatesh/KV/blob/master/xeeva_datascience_assignment_kiran_venkatesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import ModelCheckpoint
from textblob import TextBlob
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
import gensim
from gensim import corpora


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

***Task-1.1 Fuzzy Text Match***

In [ ]:
df=pd.read_excel("/content/DocumentsData.xlsx")

In [ ]:
mask = np.random.rand(len(df)) < 0.8
train=df[mask]
test=df[~mask]

In [ ]:
print("Length of train and test",len(train),len(test))

Length of train and test 388 112


In [ ]:
Best_match=[]
for test_doc in test["Doc Name"]:
  for train_doc in train["Doc Name"]:
    score=fuzz.token_sort_ratio(test_doc, train_doc)
    if score>75:
      Best_match.append([test_doc,train_doc,score])
      break


bf=pd.DataFrame(Best_match,columns=["Best_Match_Test_doc","Best_Match_Train_doc","Confidence_Match_score"])
len(bf)




77

In [ ]:
print("Count of Matched Docs:",len(bf))
print("% of docs matched in Test Docs:",(len(bf)/len(test))*100)

Count of Matched Docs: 77
% of docs matched in Test Docs: 68.75


In [ ]:
print("Matched Docs")
bf

Matched Docs


,Best_Match_Test_doc,Best_Match_Train_doc,Confidence_Match_score
0,Value Collection 12 Inch Long x 9 Inch Wide x ...,Value Collection 15 Inch Long x 12 Inch Wide x...,97
1,"3M 1-1/2"" Diam Aluminum Oxide Quick Change Dis...","3M 1"" Diam Aluminum Oxide Quick Change Disc Ty...",98
2,"3M 2"" Diam Aluminum Oxide Quick Change Disc Ty...","3M 1"" Diam Aluminum Oxide Quick Change Disc Ty...",85
3,"3M 3"" Diam Aluminum Oxide Quick Change Disc Ty...","3M 1"" Diam Aluminum Oxide Quick Change Disc Ty...",85
4,Dupont Size 3XL White Lab Coat without Pockets...,Dupont Size L White Lab Coat without Pockets T...,96
...,...,...,...
72,"5/16""-18 316 Stainless Steel Finished Hex Nut","3/8""-16 316 Stainless Steel Finished Hex Nut",94
73,"Pilot Light, Full Voltage, 120 Volt AC/DC, Red...","Pilot Light, Full Voltage, 120 Volt AC/DC, Gre...",96
74,"Type 12 Sa Disconnect Encl., Bulletin A25 (typ...","Type 12 Sa Disconnect Encl., Bulletin A25 (typ...",96
75,HyFlex® Cut Resistant Gloves - Polyurethane Co...,HyFlex® Cut Resistant Gloves - Polyurethane Co...,98


***Task-1.2 ML Model Fitting***

In [ ]:
mf=pd.read_csv("/content/TestParam.csv")

In [ ]:
mf.head()

,label,param0,param1,param2,param3,param4,param5,param6,param7,param8,param9,param10,param11,param12,param13,param14,param15,param16,param17,param18,param19,param20,param21,param22,param23,param24,param25,param26,param27,param28,param29,param30,param31,param32,param33,param34,param35,param36,param37,param38,...,param744,param745,param746,param747,param748,param749,param750,param751,param752,param753,param754,param755,param756,param757,param758,param759,param760,param761,param762,param763,param764,param765,param766,param767,param768,param769,param770,param771,param772,param773,param774,param775,param776,param777,param778,param779,param780,param781,param782,param783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x=mf.iloc[:,1:].values
y=mf.iloc[:,0].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 21)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1011
           1       0.97      0.98      0.98      1137
           2       0.91      0.94      0.93      1020
           3       0.92      0.91      0.92      1044
           4       0.93      0.93      0.93      1045
           5       0.93      0.91      0.92       973
           6       0.97      0.95      0.96      1076
           7       0.96      0.95      0.96      1124
           8       0.93      0.90      0.92      1004
           9       0.92      0.92      0.92      1066

    accuracy                           0.94     10500
   macro avg       0.94      0.94      0.94     10500
weighted avg       0.94      0.94      0.94     10500



In [ ]:
print("Accuracy of Random Forest Model in Test_Params.csv:",accuracy_score(y_test,y_pred))

Accuracy of Random Forest Model in Test_Params.csv: 0.9406666666666667


***Task-2 EDA***

In [ ]:
xls = pd.ExcelFile('/content/EmployeeData.xlsx')
df1 = pd.read_excel(xls, 'IT')
df2 = pd.read_excel(xls, 'Admin')
df3 = pd.read_excel(xls, 'Support')

In [ ]:
dr=df1.append(df2, ignore_index = True)
eda=dr.append(df3,ignore_index=True) 

In [ ]:
eda

,Name,Dept,Salary,RcvDate
0,Vertie,IT,3000,2018-08-05
1,Charlsie,IT,2000,2018-08-05
2,Manual,IT,4000,2018-08-05
3,Trudi,IT,6000,2018-08-05
4,Chong,IT,5600,2018-08-05
5,Chong,IT,5600,2018-08-06
6,Vertie,IT,3000,2018-08-06
7,Deedee,IT,20000,2018-08-06
8,Ramonita,Admin,9500,2018-08-06
9,Terica,Admin,8900,2018-08-06


In [ ]:
unique_eda=eda.drop_duplicates(['Name','Dept','Salary'], keep='first')

In [ ]:
unique_eda

,Name,Dept,Salary,RcvDate
0,Vertie,IT,3000,2018-08-05
1,Charlsie,IT,2000,2018-08-05
2,Manual,IT,4000,2018-08-05
3,Trudi,IT,6000,2018-08-05
4,Chong,IT,5600,2018-08-05
7,Deedee,IT,20000,2018-08-06
8,Ramonita,Admin,9500,2018-08-06
9,Terica,Admin,8900,2018-08-06
10,Hyman,Admin,7200,2018-08-06
11,Alysha,Admin,4400,2018-08-06


In [ ]:
unique_eda.groupby(['Dept'])['Name'].count()

Dept
Admin      11
IT          6
Support     5
Name: Name, dtype: int64

In [ ]:
unique_eda.groupby(['Dept'])['Salary'].max()

Dept
Admin       9500
IT         20000
Support     8200
Name: Salary, dtype: int64

In [ ]:
new_eda=unique_eda.groupby(
    ['Dept']
).agg(
    {
        'Salary': [min,max],
        'Name': "count",
    }
)

In [ ]:
new_eda

Salary         Name
           min    max count
Dept                       
Admin     4400   9500    11
IT        2000  20000     6
Support   4400   8200     5

***Task-3 NLP Text based Clustering***

In [ ]:
nlp=pd.read_excel("/content/DocumentsData.xlsx")
nlp.head()

,Id,Doc Name
0,1,"Nozzle, 220542"
1,2,Value Collection 15 Inch Long x 12 Inch Wide x...
2,3,Made in USA 18 Inch Long x 13 Inch Wide x 4 mi...
3,4,"8""W x 10""L 4mil Clear Reclosable Bag"
4,5,Value Collection 12 Inch Long x 9 Inch Wide x ...


In [ ]:
def textblob_tokenizer(str_input):
    str_input = re.sub(r'\d+','', str_input)
    str_input=str_input.strip()

    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens if token.isalpha()]
    lemmatizer=WordNetLemmatizer()
    words=[lemmatizer.lemmatize(word) for word in words]
    return words

In [ ]:
vec = TfidfVectorizer(stop_words='english',tokenizer=textblob_tokenizer)
X = vec.fit_transform(nlp["Doc Name"])
tf = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'el', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'le', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'plea', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'u', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  

In [ ]:
tf

,abhsp,abras,absorb,accessori,acryl,act,adhes,adjust,air,aircraft,alloy,alo,aluminum,ambidextr,american,angl,ankl,ansel,ansi,ap,appli,applic,approv,arbor,area,aspen,assembl,astm,atrax,attach,averag,awg,b,ba,bag,band,bar,batteri,bayonet,bbp,...,vac,vacuum,valeo,valu,varnish,vdc,veri,vinyl,volt,voltag,w,washer,weather,weight,weld,wet,wheel,white,whl,wide,width,wire,wood,work,worksmart,wound,wr,wrap,wrist,x,xl,xxl,y,yd,year,yellow,z,zinc,ziplock,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.255188,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.339423,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333747,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.244554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333747,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447199,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.394033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.255188,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.339423,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
en_stops = set(stopwords.words('english'))
def remove_stopwords(inp):
  out=[]
  for i in inp:
    if i not in en_stops:
      out.append(i)

  return out

In [ ]:
doc_clean=[remove_stopwords(textblob_tokenizer(doc)) for doc in nlp["Doc Name"]] 
dictionary = corpora.Dictionary(doc_clean)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=12, id2word = dictionary, passes=100)

In [ ]:
L=[]
for idx, topic in ldamodel.print_topics(-1):
    print('Topic Category: {} \nWords: {}'.format(idx, topic))


Topic Category: 0 
Words: 0.050*"steel" + 0.050*"finish" + 0.049*"screw" + 0.045*"x" + 0.040*"drive" + 0.040*"cap" + 0.039*"alloy" + 0.037*"black" + 0.037*"hex" + 0.033*"socket"
Topic Category: 1 
Words: 0.090*"x" + 0.043*"wide" + 0.040*"long" + 0.033*"hose" + 0.031*"inch" + 0.026*"l" + 0.026*"mil" + 0.024*"blade" + 0.022*"reciproc" + 0.022*"saw"
Topic Category: 2 
Words: 0.131*"drill" + 0.130*"split" + 0.130*"jobber" + 0.129*"point" + 0.129*"cobalt" + 0.129*"finish" + 0.109*"gold" + 0.024*"ltr" + 0.004*"bit" + 0.004*"sd"
Topic Category: 3 
Words: 0.077*"heavi" + 0.073*"duti" + 0.073*"straw" + 0.073*"oxid" + 0.017*"printer" + 0.014*"type" + 0.014*"ribbon" + 0.014*"label" + 0.013*"color" + 0.010*"seri"
Topic Category: 4 
Words: 0.067*"inch" + 0.039*"handl" + 0.035*"bristl" + 0.026*"plastic" + 0.022*"roller" + 0.021*"wide" + 0.020*"long" + 0.018*"brush" + 0.018*"paint" + 0.018*"c"
Topic Category: 5 
Words: 0.072*"screw" + 0.067*"steel" + 0.066*"x" + 0.061*"socket" + 0.060*"head" + 0.052*

In [ ]:
i=0
for d in doc_clean:
  print(i,d)
  i=i+1



0 ['nozzl']
1 ['valu', 'collect', 'inch', 'long', 'x', 'inch', 'wide', 'x', 'mil', 'thick', 'ziplock', 'polybag', 'heavi', 'duti', 'grade']
2 ['made', 'usa', 'inch', 'long', 'x', 'inch', 'wide', 'x', 'mil', 'thick', 'ziplock', 'reclos', 'bag', 'heavi', 'duti', 'grade']
3 ['w', 'x', 'l', 'mil', 'clear', 'reclos', 'bag']
4 ['valu', 'collect', 'inch', 'long', 'x', 'inch', 'wide', 'x', 'mil', 'thick', 'ziplock', 'polybag', 'heavi', 'duti', 'grade']
5 ['ansel', 'size', 'xl', 'mil', 'industri', 'grade', 'unpowd', 'nitril', 'dispos', 'glove', 'long', 'blue', 'textur', 'finish', 'fda', 'approv', 'bead', 'roll', 'cuff', 'static', 'dissip', 'ambidextr']
6 ['mil', 'blue', 'nitril', 'textur', 'food', 'handl', 'dispos', 'glove']
7 ['diam', 'aluminum', 'oxid', 'quick', 'chang', 'disc', 'type', 'r', 'attach', 'system', 'blue', 'veri', 'fine', 'grade', 'max', 'rpm']
8 ['diam', 'grit', 'aluminum', 'oxid', 'quick', 'chang', 'disc', 'type', 'r', 'attach', 'system', 'brown', 'fine', 'grade', 'max', 'rpm']

***Task-4 Deep Learning***

In [ ]:
dl=pd.read_csv("/content/deep_learning.csv")

In [ ]:
dl.iloc[:,0:-1]

,Number of times pregnant,glucose concentration,blood pressure,Triceps thickness,insulin level,Body mass index,Diabetes pedigree function,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [ ]:
X = dl.iloc[:,0:-1].values
Y= dl.iloc[:,-1].values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [ ]:
X

In [ ]:
model=Sequential()
model.add(Dense(12,activation='relu',input_dim=8))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,epochs=2000)

Epoch 1/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.8106 - accuracy: 0.3840
Epoch 2/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.7893 - accuracy: 0.4019
Epoch 3/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.7418 - accuracy: 0.4730
Epoch 4/2000
20/20 [==============================] - 0s 997us/step - loss: 0.7160 - accuracy: 0.5077
Epoch 5/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.6804 - accuracy: 0.5759
Epoch 6/2000
20/20 [==============================] - 0s 974us/step - loss: 0.6801 - accuracy: 0.5976
Epoch 7/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.6698 - accuracy: 0.6352
Epoch 8/2000
20/20 [==============================] - 0s 937us/step - loss: 0.6312 - accuracy: 0.7029
Epoch 9/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.6109 - accuracy: 0.7056
Epoch 10/2000
20/20 [==============================] - 0s 1ms/step - loss: 0.6165 - accuracy: 

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
rounded = [round(x[0]) for x in Y_pred]
Y_pred = rounded
Y_pred

In [ ]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       107
           1       0.72      0.60      0.65        47

    accuracy                           0.81       154
   macro avg       0.78      0.75      0.76       154
weighted avg       0.80      0.81      0.80       154



In [ ]:
model.save("ANN.h5")